# Basic Setup

Import code from either Google Colab or local drive.
Select that option by either executing the first or second cell.

In [1]:
# Google Colab setup
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/adl4cv")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
# ONLY NECESSARY FOR LOCAL EXECUTION (WORKS WITHOUT THIS CELL IN GOOGLE COLAB)
# Setup that is necessary for jupyter notebook to find sibling-directories
# see: https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# Imports for this notebook

from networks.temporal_encoder_network import TemporalEncoder
from training.solver import Solver
from training.sequence_dataloader import FaceForensicsVideosDataset, ToTensor
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler
import torch

In [3]:
# Check training on GPU?

print("Training is on GPU with CUDA: {}".format(torch.cuda.is_available()))

Training is on GPU with CUDA: True


# Load Data and Model

*   Load FaceForensics sequences: Choose a list of corresponding file-paths.
*   Load the model for this notebook.
*   Enable or disable usage of optical flow / warp as input.

In [4]:
# Decide here if using optical flow is desired
# If yes: warps will be calculated in dataloader and model/solver will get it as an input
# If no: warps will not be calculated and model/solver do not get it as an input

opticalFlowEnabled = False

print("Will use Optical Flow / Warp as input for network: {}".format(opticalFlowEnabled))

Will use Optical Flow / Warp as input for network: False


In [5]:
# Load Dataset from drive location

sequence = "sequences_299x299_10seq@10frames_skip_5_uniform"
#sequence = "sequences_299x299_10seq@5frames_skip_1_uniform"

original_location = "F:/Google Drive/FaceForensics_large/original_sequences/youtube/c40/" + sequence
deepfake_location = "F:/Google Drive/FaceForensics_large/manipulated_sequences/Deepfakes/c40/" + sequence
data_location = [original_location, deepfake_location]

data_dict = {"dataset-" + str(i): data_location[i] for i in range(len(data_location))}

dataset = FaceForensicsVideosDataset(data_location,
                                     transform=ToTensor(),
                                     num_frames=10, #num_frames=5
                                     #max_number_videos_per_directory=4,
                                     calculateOpticalFlow=opticalFlowEnabled,
                                     verbose=False,
                                    caching=False)

print("Loaded following data: {}".format(data_dict))

Loading directory 1/2: F:/Google Drive/FaceForensics_large/original_sequences/youtube/c40/sequences_299x299_10seq@10frames_skip_5_uniform



Loading directory 2/2: F:/Google Drive/FaceForensics_large/manipulated_sequences/Deepfakes/c40/sequences_299x299_10seq@10frames_skip_5_uniform



Loaded following data: {'dataset-0': 'F:/Google Drive/FaceForensics_large/original_sequences/youtube/c40/sequences_299x299_10seq@10frames_skip_5_uniform', 'dataset-1': 'F:/Google Drive/FaceForensics_large/manipulated_sequences/Deepfakes/c40/sequences_299x299_10seq@10frames_skip_5_uniform'}


In [7]:
# Setup pytorch dataloaders
# from: https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

dataset_args = {
    "batch_size": 64,
    "validation_percentage": 0.2,
    "sequence": sequence,
    **data_dict
}

# Creating data indices for training and validation splits:
train_indices, val_indices = dataset.get_train_val_lists(1 - dataset_args["validation_percentage"], dataset_args["validation_percentage"])

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

# Should set num_workers=0, otherwise the caching in the dataset does not work... but why?
train_loader = torch.utils.data.DataLoader(dataset, batch_size=dataset_args["batch_size"], 
                                           sampler=train_sampler,
                                           num_workers=32)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=dataset_args["batch_size"],
                                                sampler=valid_sampler,
                                                num_workers=32)

dataset_args["train_len"] = len(train_loader)
dataset_args["val_len"] = len(validation_loader)

print("Dataset parameters: {}".format(dataset_args))

Dataset parameters: {'batch_size': 64, 'validation_percentage': 0.2, 'sequence': 'sequences_299x299_10seq@10frames_skip_5_uniform', 'dataset-0': 'F:/Google Drive/FaceForensics_large/original_sequences/youtube/c40/sequences_299x299_10seq@10frames_skip_5_uniform', 'dataset-1': 'F:/Google Drive/FaceForensics_large/manipulated_sequences/Deepfakes/c40/sequences_299x299_10seq@10frames_skip_5_uniform', 'train_len': 249, 'val_len': 63}


In [8]:
# Load Temporal Encoder II Model

model_args={
    "model_choice": "xception",
    "num_out_classes": 2,
    "dropout": 0.5,
    "num_input_images": 10, #5
    "feature_dimension": 64,
    "temporal_encoder_depth": 5,
    "delta_t": 2,
    "useOpticalFlow": opticalFlowEnabled
}

model = TemporalEncoder(num_input_images=model_args["num_input_images"],
                        model_choice=model_args["model_choice"],
                        feature_dimension=model_args["feature_dimension"],
                        temporal_encoder_depth=model_args["temporal_encoder_depth"],
                        dropout=model_args["dropout"],
                        num_out_classes=model_args["num_out_classes"],
                        delta_t=model_args["delta_t"],
                        useOpticalFlow=model_args["useOpticalFlow"])

model_args["model"] = type(model).__name__

print("Model configuration: {}".format(model_args))

print("Only the following layers of feature extractor component (pretrained!) require gradient backpropagation (param.requires_grad)")
for name, param in model.feature_extractor.named_parameters():
    if param.requires_grad:
        print("param: {} requires_grad: {}".format(name, param.requires_grad))

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth" to C:\Users\lukas/.cache\torch\checkpoints\xception-43020ad28.pth
100%|██████████████████████████████████████████████████████████████████████████████| 87.4M/87.4M [04:17<00:00, 356kB/s]


Model configuration: {'model_choice': 'xception', 'num_out_classes': 2, 'dropout': 0.5, 'num_input_images': 10, 'feature_dimension': 64, 'temporal_encoder_depth': 5, 'delta_t': 2, 'useOpticalFlow': False, 'model': 'TemporalEncoder'}
Only the following layers of feature extractor component (pretrained!) require gradient backpropagation (param.requires_grad)
param: 23.weight requires_grad: True
param: 23.bias requires_grad: True


# Training Visualization

Start Tensorboard for visualization of the upcoming training / validation / test steps.

In [0]:
# Start tensorboard
%load_ext tensorboard
%tensorboard --logdir runs

# Training

Start training process.

In [9]:
# Create unique ID for this training process for saving to disk.

from datetime import datetime
import uuid
now = datetime.now() # current date and time
id = str(uuid.uuid1())
id_suffix = now.strftime("%Y-%b-%d_%H:%M:%S") + "_" + id

log_dir = "runs/TemporalEncoderII/" + id_suffix
print("log_dir:", log_dir)

log_dir: runs/TemporalEncoderII/2020-Jan-19_11:20:37_560791f0-3aa5-11ea-915e-d0509936bc3f


In [10]:
# Configure solver
extra_args = {
    **model_args,
    **dataset_args
}

solver = Solver(optim=torch.optim.Adam,
                optim_args={ "lr": 4e-4,
                             "betas": (0.9, 0.999),
                             "eps": 1e-8,
                             "weight_decay": 0.01}, # is the l2 regularization parameter, see: https://pytorch.org/docs/stable/optim.html
                loss_func=torch.nn.CrossEntropyLoss(),
                extra_args=extra_args,
                log_dir=log_dir)

OSError: [WinError 123] Die Syntax für den Dateinamen, Verzeichnisnamen oder die Datenträgerbezeichnung ist falsch: 'runs/TemporalEncoderII/2020-Jan-19_11:20:37_560791f0-3aa5-11ea-915e-d0509936bc3f'

In [ ]:
# Start training

'''
During the first epoch, all required images (and warps) are loaded into RAM.
This might take a while, but after the first epoch, it will be faster.
The loading is done during the first epoch and not when loading the dataloader,
because it would take ~1 hour just to load all data before we can see any 
training output. Instead, we can start right away and load just-in-time.

Note that the first epoch is only slow when a brand-new data_loader is used.
Using the same data_loader for multiple trainings keeps the images loaded.
'''

solver.train(model, train_loader, validation_loader, num_epochs=10, log_nth=10)

# Test

Test with same fake set and different fake set as for training.
Will load the data and start the training.

Visualizations can be seen in Tensorboard above.

In [12]:
# Load test data for SAME FAKE SET

same_test_data_location = ["/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform",
                 "/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/Deepfakes/c40/sequences_299x299_5seq@10frames_skip_5_uniform"]

#same_test_data_location = ["/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_10seq@5frames_skip_1_uniform",
#                 "/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/Deepfakes/c40/sequences_299x299_10seq@5frames_skip_1_uniform"]

same_test_dataset = FaceForensicsVideosDataset(same_test_data_location,
                                               transform=ToTensor(),
                                               num_frames=10, #num_frames=5
                                               #max_number_videos_per_directory=4,
                                               calculateOpticalFlow=opticalFlowEnabled,
                                               verbose=False)

same_test_indices = range(len(same_test_dataset))

same_test_sampler = SubsetRandomSampler(same_test_indices)

same_test_loader = torch.utils.data.DataLoader(same_test_dataset,
                                               batch_size=8, 
                                               sampler=same_test_sampler,
                                               num_workers=4)

print("Length of same fake test set: {}".format(len(same_test_dataset)))

Loading directory 1/2: /content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform



Loading directory 2/2: /content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/Deepfakes/c40/sequences_299x299_5seq@10frames_skip_5_uniform



Length of same fake test set: 100


In [13]:
# Start testing

solver.test(model, same_test_loader, test_prefix="Same_Fake_Method", log_nth=10)

[Iteration 1/13] TEST loss: 0.789614200592041
[Iteration 11/13] TEST loss: 0.32681384682655334
[TEST] mean acc/loss: 0.7307692307692307/0.604640007019043


In [14]:
# Load test data for DIFFERENT FAKE SET

fake_type = "NeuralTextures" # Face2Face, FaceSwap, NeuralTextures

dif_test_data_location = ["/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform",
                 "/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/" + fake_type + "/c40/sequences_299x299_5seq@10frames_skip_5_uniform"]

#dif_test_data_location = ["/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_10seq@5frames_skip_1_uniform",
#                 "/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/FaceSwap/c40/sequences_299x299_10seq@5frames_skip_1_uniform"]

dif_test_dataset = FaceForensicsVideosDataset(dif_test_data_location,
                                              transform=ToTensor(),
                                              num_frames=10,
                                              #max_number_videos_per_directory=4,
                                              calculateOpticalFlow=opticalFlowEnabled,
                                              verbose=False) #num_frames=5

dif_test_indices = range(len(dif_test_dataset))

dif_test_sampler = SubsetRandomSampler(dif_test_indices)

dif_test_loader = torch.utils.data.DataLoader(dif_test_dataset,
                                              batch_size=8, 
                                              sampler=dif_test_sampler,
                                              num_workers=4)

print("Length of dif fake test set: {}".format(len(dif_test_dataset)))

Loading directory 1/2: /content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform



Loading directory 2/2: /content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/NeuralTextures/c40/sequences_299x299_5seq@10frames_skip_5_uniform



Length of dif fake test set: 100


In [15]:
# Start testing

solver.test(model, dif_test_loader, test_prefix="Different_Fake_Method_" + fake_type, log_nth=10)

[Iteration 1/13] TEST loss: 0.4155687093734741
[Iteration 11/13] TEST loss: 0.72868812084198
[TEST] mean acc/loss: 0.5288461538461539/1.0442492961883545


In [16]:
# Load test data for DIFFERENT FAKE SET

fake_type = "Face2Face" # Face2Face, FaceSwap, NeuralTextures

dif_test_data_location = ["/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform",
                 "/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/" + fake_type + "/c40/sequences_299x299_5seq@10frames_skip_5_uniform"]

#dif_test_data_location = ["/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_10seq@5frames_skip_1_uniform",
#                 "/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/FaceSwap/c40/sequences_299x299_10seq@5frames_skip_1_uniform"]

dif_test_dataset = FaceForensicsVideosDataset(dif_test_data_location,
                                              transform=ToTensor(),
                                              num_frames=10,
                                              #max_number_videos_per_directory=4,
                                              calculateOpticalFlow=opticalFlowEnabled,
                                              verbose=False) #num_frames=5

dif_test_indices = range(len(dif_test_dataset))

dif_test_sampler = SubsetRandomSampler(dif_test_indices)

dif_test_loader = torch.utils.data.DataLoader(dif_test_dataset,
                                              batch_size=8, 
                                              sampler=dif_test_sampler,
                                              num_workers=4)

print("Length of dif fake test set: {}".format(len(dif_test_dataset)))

Loading directory 1/2: /content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform



Loading directory 2/2: /content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/Face2Face/c40/sequences_299x299_5seq@10frames_skip_5_uniform



Length of dif fake test set: 100


In [17]:
# Start testing

solver.test(model, dif_test_loader, test_prefix="Different_Fake_Method_" + fake_type, log_nth=10)

[Iteration 1/13] TEST loss: 0.6893520951271057
[Iteration 11/13] TEST loss: 0.539339542388916
[TEST] mean acc/loss: 0.5480769230769231/0.9120302796363831


In [18]:
# Load test data for DIFFERENT FAKE SET

fake_type = "FaceSwap" # Face2Face, FaceSwap, NeuralTextures

dif_test_data_location = ["/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform",
                 "/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/" + fake_type + "/c40/sequences_299x299_5seq@10frames_skip_5_uniform"]

#dif_test_data_location = ["/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_10seq@5frames_skip_1_uniform",
#                 "/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/FaceSwap/c40/sequences_299x299_10seq@5frames_skip_1_uniform"]

dif_test_dataset = FaceForensicsVideosDataset(dif_test_data_location,
                                              transform=ToTensor(),
                                              num_frames=10,
                                              #max_number_videos_per_directory=4,
                                              calculateOpticalFlow=opticalFlowEnabled,
                                              verbose=False) #num_frames=5

dif_test_indices = range(len(dif_test_dataset))

dif_test_sampler = SubsetRandomSampler(dif_test_indices)

dif_test_loader = torch.utils.data.DataLoader(dif_test_dataset,
                                              batch_size=8, 
                                              sampler=dif_test_sampler,
                                              num_workers=4)

print("Length of dif fake test set: {}".format(len(dif_test_dataset)))

Loading directory 1/2: /content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform



Loading directory 2/2: /content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/FaceSwap/c40/sequences_299x299_5seq@10frames_skip_5_uniform



Length of dif fake test set: 100


In [19]:
# Start testing

solver.test(model, dif_test_loader, test_prefix="Different_Fake_Method_" + fake_type, log_nth=10)

[Iteration 1/13] TEST loss: 2.2012734413146973
[Iteration 11/13] TEST loss: 0.7051862478256226
[TEST] mean acc/loss: 0.6538461538461539/1.229109525680542


# Save the model

Save network with its weights to disk.

See torch.save function: https://pytorch.org/docs/stable/notes/serialization.html#recommend-saving-models 

Load again with `the_model = TheModelClass(*args, **kwargs) the_model.load_state_dict(torch.load(PATH))`

In [0]:
def save_model(modelname, model):
    filepath = "/content/drive/My Drive/adl4cv/saved_results/models/" + modelname + ".pt"
    torch.save(model.state_dict(), filepath)

In [0]:
save_model("temporal_encoder_2_" + id_suffix, model)

In [22]:
# LOAD MODEL AGAIN for verification purposes
# Should print: <All keys matched successfully>
filepath = "/content/drive/My Drive/adl4cv/saved_results/models/" + "temporal_encoder_2_" + id_suffix + ".pt"
model.load_state_dict(torch.load(filepath))

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

...

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>